In [2]:
import math
from typing import Tuple

import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.nn import TransformerEncoder as TE, TransformerEncoderLayer as TEL 
from torch.utils.data import dataset

PositionalEncoding module injects some information about the relative or absolute position of the tokens in the sequence. The positional encodings have the same dimension as the embeddings so that the two can be summed. Here, we use sine and cosine functions of different frequencies.

In [3]:
class PositionalEncoding(nn.Module):

  def __init__(self, d_model: int, dropout: float = 0.1, max_len:int = 5000):
    super().__init__()
    self.dropout = nn.Dropout(p=dropout)

    position = torch.arange(max_len).unsqueeze(1)
    div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
    pe = torch.zeros(max_len, 1, d_model)
    pe[:, 0, 0::2] = torch.sin(position * div_term)
    pe[:, 0, 1::2] = torch.cos(position * div_term)
    self.register_buffer('pe', pe)

  
  def forward(self, x: Tensor) -> Tensor:
    """
    Args:
      x: Tensor, shape [seq_len, batch_size, embedding_dim]
    """
    x = x + self.pe[:x.size(0)]
    return self.dropout(x)

In [16]:
class TransformerModel(nn.Module):

  def __init__(self, ntoken: int, d_model: int, nhead: int, d_hid: int,
               nlayers: int, dropout: float = 0.5):
    super().__init__()
    self.model_type = 'Transformer'
    self.pos_encoder = PositionalEncoding(d_model, dropout)
    encoder_layers = TEL(d_model, nhead, d_hid, dropout)
    self.transformer_encoder = TE(encoder_layers, nlayers)
    self.encoder = nn.Embedding(ntoken, d_model)
    self.d_model = d_model
    self.decoder = nn.Linear(d_model, ntoken)

    self.init_weights()
  

  def init_weights(self) -> None:
    initrange = 0.1
    self.encoder.weight.data.uniform_(-initrange, initrange)
    self.decoder.bias.data.zero_()
    self.decoder.weight.data.uniform_(-initrange, initrange)
  

  def forward(self, src: Tensor, src_mask: Tensor) -> Tensor:
    """
    Args:
      src: Tensor, shape [seq_len, batch_size]
      src_mask: Tensor, shape [seq_len, seq_len]

    Returns:
      output Tensor of shape [seq_len, batch_size, ntoken]
    """

    src = self.encoder(src) * math.sqrt(self.d_model)
    src = self.pos_encoder(src)
    output = self.transformer_encoder(src, src_mask)
    output = self.decoder(output)
    return output

In [6]:
def generate_square_subsequent_mask(sz: int) -> Tensor:
  """Generates an upper-triangular matrix of -inf, with zeros on diag."""
  return torch.triu(torch.ones(sz, sz) * float('-inf'), diagonal=1)

In [7]:
from torchtext.datasets import WikiText2
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

In [8]:
train_iter = WikiText2(split='train')
tokenizer = get_tokenizer('basic_english')
vocab = build_vocab_from_iterator(map(tokenizer, train_iter), specials=['<unk>'])
vocab.set_default_index(vocab['<unk>'])

100%|██████████| 4.48M/4.48M [00:01<00:00, 2.90MB/s]


In [9]:
def data_process(raw_text_iter: dataset.IterableDataset) -> Tensor:
  """Converts raw text into a flat Tensor."""
  data = [torch.tensor(vocab(tokenizer(item)), dtype=torch.long) for item in raw_text_iter]
  return torch.cat(tuple(filter(lambda t: t.numel() > 0, data)))

In [10]:
# train_iter was "consumed" by the process of building the vocab, so we have to create it again
train_iter, val_iter, test_iter = WikiText2()
train_data = data_process(train_iter)
val_data = data_process(val_iter)
test_data = data_process(test_iter)

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
def batchify(data: Tensor, bsz: int) -> Tensor:
  """Divides the data into bsz separate sequences, removing extra elements that wouldn't cleanly fit.

  Args:
    data: Tensor, shape [N]
    bsz: int, batch size

  Returns:
    Tensor of shape [N // bsz, bsz]
  """

  seq_len = data.size(0) // bsz
  data = data[:seq_len * bsz]
  data = data.view(bsz, seq_len).t().contiguous()
  return data.to(device)

In [13]:
batch_size = 20
eval_batch_size = 10
train_data = batchify(train_data, batch_size)
val_data = batchify(val_data, eval_batch_size)
test_data = batchify(test_data, eval_batch_size)

In [23]:
bptt = 35
def get_batch(source: Tensor, i: int) -> Tuple[Tensor, Tensor]:
  """
  Args:
    source: Tensor, shape [full_seq_len, batch_size]
    i: int

  Returns:
    tuple (data, target), where data has shape [seq_len, batch_size] and
    target has shape [seq_len * batch_size]
  """

  seq_len = min(bptt, len(source) - 1 - i)
  data = source[i:i+seq_len]
  target = source[i+1:i+1+seq_len].reshape(-1)
  return data, target


In [24]:
ntokens = len(vocab)  # size of vocabulary
emsize = 200  # embedding dimension
d_hid = 200  # dimension of the feedforward network model in nn.TransformerEncoder
nlayers = 2  # number of nn.TransformerEncoderLayer in nn.TransformerEncoder
nhead = 2  # number of heads in nn.MultiheadAttention
dropout = 0.2  # dropout probability
model = TransformerModel(ntokens, emsize, nhead, d_hid, nlayers, dropout).to(device)

In [25]:
import copy
import time

In [26]:
criterion = nn.CrossEntropyLoss()
lr = 5.0
optimizer = torch.optim.SGD(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.95)

In [27]:
def train(model: nn.Module) -> None:
  
  model.train()
  total_loss = 0
  log_interval = 200
  start_time = time.time()
  src_mask = generate_square_subsequent_mask(bptt).to(device)

  num_batches = len(train_data) // bptt
  for batch, i in enumerate(range(0, train_data.size(0) - 1, bptt)):
    data, targets = get_batch(train_data, i)
    batch_size = data.size(0)
    if batch_size != bptt: # Only on last batch
      src_mask = src_mask[:batch_size, :batch_size]
    output = model(data, src_mask)
    loss = criterion(output.view(-1, ntokens), targets)

    optimizer.zero_grad()
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
    optimizer.step()

    total_loss += loss.item()

    if batch % log_interval == 0 and batch > 0:
      lr = scheduler.get_last_lr()[0]
      ms_per_batch = (time.time() - start_time) * 1000 / log_interval
      cur_loss = total_loss / log_interval
      ppl = math.exp(cur_loss)
      print(f'| Epoch {epoch:3d} | batch [{batch:5d}/{num_batches:5d}] | '
            f'lr {lr:02.2f} | ms/batch {ms_per_batch:5.2f} | '
            f'loss: {cur_loss:5.2f} | ppl {ppl:8.2f}'
      )
      total_loss = 0
      start_time = time.time()

In [28]:
def evaluate(model: nn.Module, eval_data: Tensor) -> float:
  
  model.eval()
  total_loss = 0
  src_mask = generate_square_subsequent_mask(bptt).to(device)

  with torch.no_grad():
  
    total_loss = 0
  
    for i in range(0, eval_data.size(0) - 1, bptt):
      data, targets = get_batch(eval_data, i)
      batch_size = data.size(0)
      if batch_size != bptt:
        src_mask = src_mask[:batch_size, :batch_size]
      output = model(data, src_mask)
      output_flat = output.view(-1, ntokens)
      total_loss += batch_size * criterion(output_flat, targets).item()
  
  return total_loss / (len(eval_data) - 1)

In [29]:
best_val_loss = float('inf')
epochs = 10
best_model = None
for epoch in range(1, epochs + 1):
  epoch_start_time = time.time()
  train(model)
  val_loss = evaluate(model, val_data)
  val_ppl = math.exp(val_loss)
  elapsed = time.time() - epoch_start_time
  print('-' * 89)
  print(f'| end of epoch {epoch:3d} | time: {elapsed:5.2f}s | '
        f'valid loss {val_loss:5.2f} | valid ppl {val_ppl:8.2f}')
  print('-' * 89)

  if val_loss < best_val_loss:
      best_val_loss = val_loss
      best_model = copy.deepcopy(model)

  scheduler.step()

| Epoch   1 | batch [  200/ 2928] | lr 5.00 | ms/batch 38.97 | loss:  8.24 | ppl  3785.91
| Epoch   1 | batch [  400/ 2928] | lr 5.00 | ms/batch 37.82 | loss:  6.91 | ppl  1000.46
| Epoch   1 | batch [  600/ 2928] | lr 5.00 | ms/batch 37.88 | loss:  6.46 | ppl   640.40
| Epoch   1 | batch [  800/ 2928] | lr 5.00 | ms/batch 37.83 | loss:  6.31 | ppl   547.71
| Epoch   1 | batch [ 1000/ 2928] | lr 5.00 | ms/batch 37.71 | loss:  6.19 | ppl   488.78
| Epoch   1 | batch [ 1200/ 2928] | lr 5.00 | ms/batch 37.74 | loss:  6.16 | ppl   474.24
| Epoch   1 | batch [ 1400/ 2928] | lr 5.00 | ms/batch 38.14 | loss:  6.11 | ppl   449.03
| Epoch   1 | batch [ 1600/ 2928] | lr 5.00 | ms/batch 37.81 | loss:  6.11 | ppl   448.64
| Epoch   1 | batch [ 1800/ 2928] | lr 5.00 | ms/batch 37.65 | loss:  6.03 | ppl   413.77
| Epoch   1 | batch [ 2000/ 2928] | lr 5.00 | ms/batch 37.76 | loss:  6.02 | ppl   413.40
| Epoch   1 | batch [ 2200/ 2928] | lr 5.00 | ms/batch 37.76 | loss:  5.90 | ppl   363.39
| Epoch   

In [30]:
test_loss = evaluate(best_model, test_data)
test_ppl = math.exp(test_loss)
print('=' * 89)
print(f'| End of training | test loss {test_loss:5.2f} | '
      f'test ppl {test_ppl:8.2f}')
print('=' * 89)

| End of training | test loss  5.40 | test ppl   221.78
